In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import time

In [2]:
import os
os.listdir() 

['00_query.txt', '01_Transformacion_Modeling.py', 'fetch_data.ipynb']

In [3]:
def build_query(year, month):

    # Build start and end timestamps for the given month
    start = datetime(year, month, 1)
    if month == 12:
        end = datetime(year + 1, 1, 1)
    else:
        end = datetime(year, month + 1, 1)

    start_ts = start.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_ts = end.strftime("%Y-%m-%dT%H:%M:%SZ")

    query = f''' 
    select 
    matches.match_id, matches.game_mode,
    matches.radiant_team_id, matches.dire_team_id, matches.radiant_win,
    matches.start_time, matches.duration,
    ((player_matches.player_slot < 128) = matches.radiant_win) win,
    player_matches.hero_id,
    heroes.name, heroes.primary_attr, heroes.attack_type, heroes.roles,
    player_matches.kills, player_matches.deaths, player_matches.assists, 
    player_matches.gold, player_matches.last_hits, player_matches.denies, 
    player_matches.gold_per_min, player_matches.xp_per_min,
    player_matches.towers_killed, player_matches.observers_placed,
    player_matches.account_id,
    leagues.name leaguename
    FROM matches
    LEFT JOIN match_patch using(match_id)
    LEFT JOIN leagues using(leagueid)
    LEFT JOIN player_matches using(match_id)
    LEFT JOIN heroes on heroes.id = player_matches.hero_id
    LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
    LEFT JOIN teams using(team_id)
    WHERE matches.start_time >= EXTRACT(EPOCH FROM TIMESTAMP '{start_ts}')
      AND matches.start_time < EXTRACT(EPOCH FROM TIMESTAMP '{end_ts}')
    ORDER BY matches.match_id NULLS LAST '''

    query.replace(' ', '%20').replace('\n', '%0A')

    return query

In [4]:
def build_api_url(year, month):
    base_url = "https://api.opendota.com/api/explorer"
    query = build_query(year, month)  # Example for the year 2023
    url = f"{base_url}?sql={query}"
    return url

In [5]:
def fetch_data_to_df(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "rows" in data:
            return pd.DataFrame(data["rows"])
        else:
            raise ValueError("Key 'rows' not found in JSON response.")
    else:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

In [10]:
def fetch_and_save_all(start_year, end_year, output_path):
    is_first_chunk = True

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            try:
                print(f"Fetching {year}-{month:02d}...")
                url = build_api_url(year, month)
                df = fetch_data_to_df(url)

                # Append to CSV
                df.to_csv(output_path, mode='a', index=False, header=is_first_chunk, encoding='utf-8-sig', sep= '|')
                is_first_chunk = False  # Only write header on the first write

                time.sleep(1)  # delay to avoid rate limits
            except Exception as e:
                print(f"Error for {year}-{month:02d}: {e}")

In [11]:
fetch_and_save_all(2017, 2025, "../data/bronze/all_dota_matches.csv")


Fetching 2017-01...
Fetching 2017-02...
Fetching 2017-03...
Fetching 2017-04...
Fetching 2017-05...
Fetching 2017-06...
Fetching 2017-07...
Fetching 2017-08...
Fetching 2017-09...
Fetching 2017-10...
Fetching 2017-11...
Fetching 2017-12...
Fetching 2018-01...
Fetching 2018-02...
Fetching 2018-03...
Fetching 2018-04...
Fetching 2018-05...
Fetching 2018-06...
Fetching 2018-07...
Fetching 2018-08...
Fetching 2018-09...
Fetching 2018-10...
Fetching 2018-11...
Fetching 2018-12...
Fetching 2019-01...
Fetching 2019-02...
Fetching 2019-03...
Fetching 2019-04...
Fetching 2019-05...
Fetching 2019-06...
Fetching 2019-07...
Fetching 2019-08...
Fetching 2019-09...
Fetching 2019-10...
Fetching 2019-11...
Fetching 2019-12...
Fetching 2020-01...
Fetching 2020-02...
Fetching 2020-03...
Fetching 2020-04...
Fetching 2020-05...
Fetching 2020-06...
Fetching 2020-07...
Fetching 2020-08...
Fetching 2020-09...
Fetching 2020-10...
Fetching 2020-11...
Fetching 2020-12...
Fetching 2021-01...
Fetching 2021-02...
